In [1]:
import json
import requests
import pprint

### Text to Triples Main Service

The following code cell demonstrates how to interact with the main Text to Triples (_t2t_) service. _t2t_ parses a blob of text, extracts relevant scientific concepts and equations and adds them to a "local" semantic graph whose [URI](https://en.wikipedia.org/wiki/Uniform_Resource_Identifier) is provided by the user. The service response includes information about the number of scientific concept and equation it was able to extract from user-specified text. 

Request URL: http://localhost:4200/darpa/aske/text2triples

Request Payload: { 'text': 'The force is defined as mass into acceleration', 'localityURI': 'http://speed-of-sound' }

This service can be tested via its Swagger Documentation as well: http://localhost:4200/darpa/aske/ui/#!/text2triples/app_text_to_triples_text_to_triples

In [7]:
text_to_triples_service_url = 'http://localhost:4200/darpa/aske/text2triples'

file_path = 'data/speed-of-sound.txt'

text = ''
with open(file_path) as f:
    text = f.read()

locality = 'http://speed-of-sound'

input_info = {'localityURI': locality, 'text': text}
headers = {'Content-Type': 'application/json'}

input_info_json = (json.dumps(input_info))

r = requests.post(text_to_triples_service_url, input_info_json, headers=headers)
pprint.pprint(r.json())

{'numConceptsExtracted': 30, 'numEquationsExtracted': 2}


### Locality Search

One of the main objectives of _t2t_ is to augment scientific models that are extracted from code. This can be accomplished by identifying scientific models that appear in text only or by identifying information about scientific models from text that can help extract better models from code. Locality Search assists in the latter (see the following code block for example usage of locality search). 

The beta release of locality search allows to search using variable names that appear in code. For instance, an ambiguous variable "a" appears in the code and one wishes to get more context about. Assuming the availability of text associated with code (and it's extraction into a local graph using _t2t_), with the help of locality search API, users can look up which scientific models (aka equations) does "a" appear in.

The response from locality search includes the equation in which the variable appears and wherever possible, the scientific concept assciated with the variable as well as scientific concept's augmented semantic type.

Request URL: http://localhost:4200/darpa/aske/equationVariableContext

Request Payload: { 'variableName': 'a', 'localityURI': 'http://speed-of-sound' }

This service can be tested via its Swagger Documentation as well: http://localhost:4200/darpa/aske/ui/#!/text2triples/app_text_to_triples_get_equation_var_context


In [8]:
# The following code block assumes contents of "speed of sound" have been processed and extracted 
# into "http://speed-of-sound"

locality_search_service_url = 'http://localhost:4200/darpa/aske/equationVariableContext'

input_info = {'localityURI': 'http://speed-of-sound', 'variableName': 'T'}
headers = {'Content-Type': 'application/json'}

input_info_json = (json.dumps(input_info))

r = requests.post(locality_search_service_url, input_info_json, headers=headers)

pprint.pprint(r.json())


{'message': 'Search Results from Graph http://speed-of-sound',
 'results': [{'entityLabel': 'color temperature',
              'entityURI': 'http://www.wikidata.org/entity/Q327408',
              'equationString': 'a^2 = R * T * {1 + (gamma - 1) / ( 1 + '
                                '(gamma-1) * [(theta/T)^2 * e^(theta/T) '
                                '/(e^(theta/T) -1)^2]) }',
              'variableName': 'T'},
             {'entityLabel': 'temperature',
              'entityURI': 'http://www.wikidata.org/entity/Q327408',
              'equationString': 'a^2 = R * T * {1 + (gamma - 1) / ( 1 + '
                                '(gamma-1) * [(theta/T)^2 * e^(theta/T) '
                                '/(e^(theta/T) -1)^2]) }',
              'variableName': 'T'},
             {'entityLabel': 'color temperature',
              'entityURI': 'http://www.wikidata.org/entity/Q327408',
              'equationString': 'a^2 = R * T * gamma',
              'variableName': 'T'},
         

### Units-based Locality Search

Additional context about the scientific model represented in the code can be obtained by parsing the surrounding comments. These comments often include refrences to the unit associated with the variable in question. Units-based locality search, allows the user to use code comments that potentially contain references to units to identify related concepts and equations from the associated local graph.

While this feature is being made available as part of the beta release, we expect the reponse of this service to evovle. The current response inlcudes the extracted units and it's associated concept, along with a list of equations (equation URIs) in which the concept appears and the variable name associated with the concept.

Request URL: http://localhost:4200/darpa/aske/unitextraction

Request Payload: { 'text': 'variable measurement degree Celsius', 'localityURI': 'http://speed-of-sound' }

This service can be tested via its Swagger Documentation as well: http://localhost:4200/darpa/aske/ui/#!/text2triples/app_text_to_triples_get_units_info

In [9]:
# The following code block assumes contents of "speed of sound" have been processed and extracted 
# into "http://speed-of-sound"

units_search_service_url = 'http://localhost:4200/darpa/aske/unitextraction'

input_info = { 'text': 'variable measurement degree Celsius', 'localityURI': 'http://speed-of-sound' }
headers = {'Content-Type': 'application/json'}

input_info_json = (json.dumps(input_info))

r = requests.post(units_search_service_url, input_info_json, headers=headers)

pprint.pprint(r.json())

[{'end': 35,
  'relatedConceptName': 'temperature',
  'relatedConceptURI': 'http://purl.obolibrary.org/obo/UO_0000005',
  'start': 21,
  'unitName': 'degree Celsius',
  'unitText': 'degree Celsius',
  'unitURI': 'http://purl.obolibrary.org/obo/UO_0000027',
  'unit_context': [{'entity': 'http://www.wikidata.org/entity/Q327408',
                    'equation': 'http://speed-of-sound#equation_2',
                    'variableName': 'T'},
                   {'entity': 'http://www.wikidata.org/entity/Q327408',
                    'equation': 'http://speed-of-sound#equation_1',
                    'variableName': 'T'},
                   {'entity': 'http://www.wikidata.org/entity/Q11466'}]}]


### Save Graph

The local semantic graph generated by _t2t_ can be exported using this API. The save graph API returns the local graph as a seralized string in the [N3](https://en.wikipedia.org/wiki/Notation3) format.

Request URL: http://localhost:4200/darpa/aske/saveGraph

Request Payload: { 'localityURI': 'http://speed-of-sound' }

This service can be tested via its Swagger Documentation as well: http://localhost:4200/darpa/aske/ui/#!/text2triples/app_text_to_triples_save_graph


In [5]:
# The following code block assumes contents of "speed of sound" have been processed and extracted 
# into "http://speed-of-sound"

save_graph_service_url = 'http://localhost:4200/darpa/aske/saveGraph'

input_info = { 'localityURI': 'http://speed-of-sound' }
headers = {'Content-Type': 'application/json'}

input_info_json = (json.dumps(input_info))

r = requests.post(save_graph_service_url, input_info_json, headers=headers)

pprint.pprint(r.json())


{'serializationFormat': 'n3',
 'triples': "b'@prefix ns1: <http://sadl.org/sadlimplicitmodel#> .\\n@prefix "
            'ns2: <http://www.wikidata.org/prop/direct/> .\\n@prefix rdf: '
            '<http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\\n@prefix rdfs: '
            '<http://www.w3.org/2000/01/rdf-schema#> .\\n@prefix xml: '
            '<http://www.w3.org/XML/1998/namespace> .\\n@prefix xsd: '
            '<http://www.w3.org/2001/XMLSchema#> '
            '.\\n\\n<http://speed-of-sound#equation_1> a ns1:ExternalEquation '
            ';\\n    ns1:arguments [ rdf:first [ a ns1:DataDescriptor '
            ';\\n                    ns1:dataType xsd:decimal '
            ';\\n                    ns1:localDescriptorName "R" ] '
            ';\\n            rdf:rest [ rdf:first [ a ns1:DataDescriptor '
            ';\\n                            ns1:augmentedType [ a '
            'ns1:SemanticType ;\\n                                    '
            'ns1:semType <http://www.wiki

### Clear Graph

The local semantic graph generated by _t2t_ can be cleared/discarded using this API.

Request URL: http://localhost:4200/darpa/aske/clearGraph

Request Payload: { 'localityURI': 'http://speed-of-sound' }

This service can be tested via its Swagger Documentation as well: http://localhost:4200/darpa/aske/ui/#!/text2triples/app_text_to_triples_clear_graph

In [6]:
clear_graph_service_url = 'http://localhost:4200/darpa/aske/clearGraph'

input_info = { 'localityURI': 'http://speed-of-sound' }
headers = {'Content-Type': 'application/json'}

input_info_json = (json.dumps(input_info))

r = requests.post(clear_graph_service_url, input_info_json, headers=headers)

pprint.pprint(r.json())

{'message': 'Graph cleared'}
